# Workshop Azure Databricks
## 05. Modèle de régression
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## Creating a Regression Model

In this exercise, you will implement a regression model that uses features of a flight to predict how late or early it will arrive.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [0]:
import datetime
now = datetime.datetime.now()
print(now)

2021-02-10 13:55:07.521275

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights. The data includes specific characteristics (or *features*) for each flight, as well as a *label* column indicating how many minutes late or early the flight arrived.

You will load this data into a DataFrame and display it.

In [0]:
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)


### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [0]:
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

train_rows = train.count()
test_rows = test.count()

print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 446275 Testing Rows: 190808

In [0]:
display(train)

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
1,1,9E,10397,12264,94,113,1
1,1,9E,10397,13851,2,2,0
1,1,9E,10529,14492,20,41,1
1,1,9E,10721,11193,-5,-2,0
1,1,9E,10721,12478,25,30,1
1,1,9E,10721,12478,37,30,1
1,1,9E,11066,10721,13,7,0
1,1,9E,11066,12478,96,82,1
1,1,9E,11193,10529,16,86,1
1,1,9E,11193,10821,62,56,1


In [0]:
display(test)

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
1,1,9E,11066,13487,-1,-12,0
1,1,9E,11193,10529,31,27,1
1,1,9E,11193,11278,-3,62,1
1,1,9E,11193,13198,4,21,0
1,1,9E,11193,13930,-6,-19,0
1,1,9E,11433,10423,-3,-10,0
1,1,9E,13930,13487,59,36,1
1,1,9E,13931,10721,-11,-25,0
1,1,9E,13931,10721,49,38,1
1,1,AA,10721,13303,-8,-11,0


### Prepare the Training Data
To train the regression model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **StringIndexer** class to generate a numeric category for each discrete **Carrier** string value, and then use the **VectorAssembler** class to transform the numeric features that would be available for a flight that hasn't yet arrived into a vector, and then rename the **ArrDelay** column to **label** as this is what we're going to try to predict.

*Note: This is a deliberately simple example. In reality you'd likely perform mulitple data preparation steps, and later in this course we'll examine how to encapsulate these steps in to a pipeline. For now, we'll just use the numeric features as they are to dewfine the traaining dataset.*

In [0]:
# Carrier is a string, and we need our features to be numeric - so we'll generate a numeric index for each distinct carrier string, and transform the dataframe to add that as a column

carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
numTrain = carrierIndexer.fit(train).transform(train)

# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "CarrierIdx", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(numTrain).select(col("features"), (col("ArrDelay").cast("Int").alias("label")))

training.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[1.0,1.0,10.0,103...| 113|
[1.0,1.0,10.0,103...| 2|
[1.0,1.0,10.0,105...| 41|
[1.0,1.0,10.0,107...| -2|
[1.0,1.0,10.0,107...| 30|
[1.0,1.0,10.0,107...| 30|
[1.0,1.0,10.0,110...| 7|
[1.0,1.0,10.0,110...| 82|
[1.0,1.0,10.0,111...| 86|
[1.0,1.0,10.0,111...| 56|
[1.0,1.0,10.0,111...| 55|
[1.0,1.0,10.0,111...| -4|
[1.0,1.0,10.0,111...| -10|
[1.0,1.0,10.0,111...| 44|
[1.0,1.0,10.0,111...| 54|
[1.0,1.0,10.0,112...| 50|
[1.0,1.0,10.0,114...| 26|
[1.0,1.0,10.0,114...| 37|
[1.0,1.0,10.0,123...| 28|
[1.0,1.0,10.0,123...| -14|
+--------------------+-----+
only showing top 20 rows

### Train a Regression Model
Next, you need to train a regression model using the training data. To do this, create an instance of the regression algorithm you want to use and use its **fit** method to train a model based on the training DataFrame. In this exercise, you will use a *Linear Regression* algorithm - though you can use the same technique for any of the regression algorithms supported in the spark.ml API.

In [0]:
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)

model = lr.fit(training)

print ("Model trained!")

Model trained!

### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **ArrDelay** column to **trueLabel**.

In [0]:
# Transform the test data to add the numeric carrier index
numTest = carrierIndexer.fit(test).transform(test)

# Generate the features vector and label
testing = assembler.transform(numTest).select(col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
testing.show()

+--------------------+---------+
 features|trueLabel|
+--------------------+---------+
[1.0,1.0,10.0,110...| -12|
[1.0,1.0,10.0,111...| 27|
[1.0,1.0,10.0,111...| 62|
[1.0,1.0,10.0,111...| 21|
[1.0,1.0,10.0,111...| -19|
[1.0,1.0,10.0,114...| -10|
[1.0,1.0,10.0,139...| 36|
[1.0,1.0,10.0,139...| -25|
[1.0,1.0,10.0,139...| 38|
[1.0,1.0,2.0,1072...| -11|
[1.0,1.0,2.0,1082...| 49|
[1.0,1.0,2.0,1129...| -8|
[1.0,1.0,2.0,1129...| 40|
[1.0,1.0,2.0,1129...| 73|
[1.0,1.0,2.0,1129...| 4|
[1.0,1.0,2.0,1129...| 12|
[1.0,1.0,2.0,1129...| -2|
[1.0,1.0,2.0,1129...| 74|
[1.0,1.0,2.0,1129...| 26|
[1.0,1.0,2.0,1129...| -11|
+--------------------+---------+
only showing top 20 rows

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict arrival delay for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted number of minutes late or early to the actual arrival delay.

In [0]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "trueLabel")

predicted.show()

+--------------------+-------------------+---------+
 features| prediction|trueLabel|
+--------------------+-------------------+---------+
[1.0,1.0,10.0,110...| 3.435832350413694| -12|
[1.0,1.0,10.0,111...| 34.77161853946348| 27|
[1.0,1.0,10.0,111...| 2.465384698627574| 62|
[1.0,1.0,10.0,111...| 8.240015003798579| 21|
[1.0,1.0,10.0,111...|-1.4780885070436618| -19|
[1.0,1.0,10.0,114...| 2.784556558269001| -10|
[1.0,1.0,10.0,139...| 59.384731105564704| 36|
[1.0,1.0,10.0,139...| -5.307498737058527| -25|
[1.0,1.0,10.0,139...| 51.14486466748583| 38|
[1.0,1.0,2.0,1072...| -5.476927129947281| -11|
[1.0,1.0,2.0,1082...| 27.43603911641658| 49|
[1.0,1.0,2.0,1129...| 8.930686472745544| -8|
[1.0,1.0,2.0,1129...| 43.262007016281196| 40|
[1.0,1.0,2.0,1129...| 74.24529688482366| 73|
[1.0,1.0,2.0,1129...| 9.913888252386473| 4|
[1.0,1.0,2.0,1129...| 24.889662467370115| 12|
[1.0,1.0,2.0,1129...|0.24058513134704462| -2|
[1.0,1.0,2.0,1129...| 72.68778483384564| 74|
[1.0,1.0,2.0,1129...| 33.98013787578613| 26|
[1.0,1.0,2.0,1129...| 0.9266028071042545| -11|
+--------------------+-------------------+---------+
only showing top 20 rows

In [0]:
display(predicted)

features,prediction,trueLabel
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11066.0, 13487.0, -1.0))",3.435832350413694,-12
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11193.0, 10529.0, 31.0))",34.77161853946348,27
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11193.0, 11278.0, -3.0))",2.465384698627574,62
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11193.0, 13198.0, 4.0))",8.240015003798579,21
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11193.0, 13930.0, -6.0))",-1.4780885070436618,-19
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11433.0, 10423.0, -3.0))",2.784556558269001,-10
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 13930.0, 13487.0, 59.0))",59.384731105564704,36
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 13931.0, 10721.0, -11.0))",-5.307498737058527,-25
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 13931.0, 10721.0, 49.0))",51.14486466748583,38
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 2.0, 10721.0, 13303.0, -8.0))",-5.476927129947281,-11


> You can open Lab06